In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [24]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [57]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [58]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' as column_title, count(*) AS count FROM boston WHERE CRIM is NULL
        UNION
    SELECT 'ZN' as column_title, count(*) AS count FROM boston WHERE ZN is NULL
        UNION
    SELECT 'INDUS' as column_title, count(*) AS count FROM boston WHERE INDUS is NULL
        UNION
    SELECT 'CHAS' as column_title, count(*) AS count FROM boston WHERE CHAS is NULL
        UNION
    SELECT 'NOX' as column_title, count(*) AS count FROM boston WHERE NOX is NULL
    ''',
    con,
)

,column_title,count
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [59]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' as column_title, count(distinct(CRIM)) AS count FROM boston
        UNION
    SELECT 'ZN' as column_title, count(distinct(ZN)) AS count FROM boston
        UNION
    SELECT 'INDUS' as column_title, count(distinct(INDUS)) AS count FROM boston
        UNION
    SELECT 'CHAS' as column_title, count(distinct(CHAS)) AS count FROM boston
        UNION
    SELECT 'NOX' as column_title, count(distinct(NOX)) AS count FROM boston
    ''',
    con,
)

,column_title,count
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [60]:
tasks+=1
pd.read_sql(
    '''
SELECT col_name
FROM (
    SELECT 'CRIM' as col_name, MIN(CRIM) AS min_val,
            (SELECT CRIM FROM boston 
             ORDER BY CRIM 
             LIMIT 1
             OFFSET (SELECT (COUNT(*)) / 2 FROM boston)) AS median_val
             FROM boston
    UNION
    SELECT 'ZN' as col_name, MIN(ZN) AS min_val,
        (SELECT ZN FROM boston 
         ORDER BY ZN 
         LIMIT 1
         OFFSET (SELECT (COUNT(*)) / 2 FROM boston)) AS median_val
         FROM boston
    UNION
    SELECT 'INDUS' as col_name, MIN(INDUS) AS min_val,
        (SELECT ZN FROM boston 
         ORDER BY INDUS 
         LIMIT 1
         OFFSET (SELECT (COUNT(*)) / 2 FROM boston)) AS median_val
         FROM boston
    UNION
    SELECT 'CHAS' as col_name, MIN(CHAS) AS min_val,
        (SELECT CHAS FROM boston 
         ORDER BY CHAS 
         LIMIT 1
         OFFSET (SELECT (COUNT(*)) / 2 FROM boston)) AS median_val
         FROM boston
    UNION
    SELECT 'NOX' as col_name, MIN(NOX) AS min_val,
        (SELECT NOX FROM boston 
         ORDER BY NOX 
         LIMIT 1
         OFFSET (SELECT (COUNT(*)) / 2 FROM boston)) AS median_val
         FROM boston)
WHERE min_val = median_val

    ''',
    con,
)

,col_name
0,CHAS
1,ZN


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [50]:

pd.read_sql(
    '''
    select RM, MEDV, ((SELECT MAX(MEDV) FROM boston) - MEDV) AS diff FROM boston WHERE MEDV in (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 25) ORDER BY MEDV LIMIT 25
    
    ''',
    con,
)

,RM,MEDV,diff
0,5.453,5.0,45.0
1,5.683,5.0,45.0
2,5.987,5.6,44.4
3,5.852,6.3,43.7
4,4.519,7.0,43.0
5,5.414,7.0,43.0
6,5.277,7.2,42.8
7,6.343,7.2,42.8
8,6.434,7.2,42.8
9,5.000,7.4,42.6


In [61]:
tasks+=1
pd.read_sql(
    '''
SELECT '25' AS num_houses, AVG(RM) AS avg_rm_bottom, 
    (SELECT RM FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) AS avg_rm_top, 
    (SELECT MEDV FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) - AVG(MEDV) AS MEDV_diff
FROM boston
WHERE MEDV IN (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 25)

UNION

SELECT '50' AS num_houses, AVG(RM) AS avg_rm_bottom, 
    (SELECT AVG(RM) FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) AS avg_rm_top, 
    (SELECT MEDV FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) - AVG(MEDV) AS MEDV_diff
FROM boston
WHERE MEDV IN (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 50)

UNION

SELECT '100' AS num_houses, AVG(RM) AS avg_rm_bottom, 
    (SELECT AVG(RM) FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) AS avg_rm_top, 
    (SELECT MEDV FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) - AVG(MEDV) AS MEDV_diff
FROM boston
WHERE MEDV IN (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 100)

UNION

SELECT '200' AS num_houses, AVG(RM) AS avg_rm_bottom, 
    (SELECT AVG(RM) FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) AS avg_rm_top, 
    (SELECT MEDV FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) - AVG(MEDV) AS MEDV_diff
FROM boston
WHERE MEDV IN (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 200)

UNION

SELECT '300' AS num_houses, AVG(RM) AS avg_rm_bottom, 
    (SELECT AVG(RM) FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) AS avg_rm_top, 
    (SELECT MEDV FROM boston WHERE MEDV = (SELECT MAX(MEDV) FROM boston)) - AVG(MEDV) AS MEDV_diff
FROM boston
WHERE MEDV IN (SELECT MEDV FROM boston ORDER BY MEDV ASC LIMIT 300)

    ''',
    con,
)

,num_houses,avg_rm_bottom,avg_rm_top,MEDV_diff
0,100,5.882832,7.484,38.149505
1,200,5.913284,7.484,35.133831
2,25,5.719000,7.489,41.977778
3,300,5.973884,7.484,33.034768
4,50,5.757608,7.484,40.290196


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [62]:
tasks+=1
pd.read_sql(
    '''
SELECT LSTAT, MEDV, DENSE_RANK() OVER (ORDER BY LSTAT) AS rank
FROM boston
WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
ORDER BY LSTAT
       ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [63]:
tasks+=1
pd.read_sql(
    '''
SELECT 
    CASE WHEN CHAS = 1 THEN 'граничит' ELSE 'не граничит' END AS 'река', 
    AVG(MEDV) AS 'средняя стоимость'
FROM 
    boston
GROUP BY 
    CHAS;

    ''',
    con,
)

,река,средняя стоимость
0,не граничит,22.093843
1,граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [64]:
tasks+=1
pd.read_sql(
    '''
SELECT CRIM, ZN, INDUS, CHAS, NOX
FROM boston
GROUP BY CRIM, ZN, INDUS, CHAS, NOX
HAVING AVG(CASE WHEN CHAS = 1 THEN 1 ELSE 0 END) > AVG(CASE WHEN CHAS = 0 THEN 1 ELSE 0 END)
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0.01501,90.0,1.21,1.0,0.4010
1,0.04560,0.0,13.89,1.0,0.5500
2,0.05644,40.0,6.41,1.0,0.4470
3,0.06127,40.0,6.41,1.0,0.4470
4,0.06129,20.0,3.33,1.0,0.4429
5,0.09065,20.0,6.96,1.0,0.4640
6,0.10469,40.0,6.41,1.0,0.4470
7,0.11069,0.0,13.89,1.0,0.5500
8,0.11425,0.0,13.89,1.0,0.5500
9,0.13587,0.0,10.59,1.0,0.4890


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [65]:
tasks+=1
pd.read_sql(
    '''
WITH p1 AS (
    SELECT INDUS, NTILE(100) OVER (ORDER BY INDUS) AS percentile
    FROM boston
), p2 AS (
    SELECT NOX, NTILE(100) OVER (ORDER BY NOX) AS percentile
    FROM boston
)
SELECT p1.percentile, p1.INDUS, p2.NOX
FROM p1 JOIN p2 ON p1.percentile = p2.percentile
WHERE p1.percentile IN (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
GROUP BY p1.percentile

    ''',
    con,
)

,percentile,INDUS,NOX
0,10,2.93,0.428
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.07,0.507
4,50,9.69,0.538
5,60,12.83,0.573
6,70,18.10,0.605
7,80,18.10,0.659
8,90,19.58,0.713
9,100,27.74,0.871


In [41]:
tasks+=1
pd.read_sql(
    '''
WITH p1 AS (
    SELECT INDUS, NTILE(100) OVER (ORDER BY INDUS) AS percentile
    FROM boston
), p2 AS (
    SELECT NOX, NTILE(100) OVER (ORDER BY NOX) AS percentile
    FROM boston
)
SELECT p1.percentile, p1.INDUS, p2.NOX
FROM p1 JOIN p2 ON p1.percentile = p2.percentile
WHERE p1.percentile IN (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
GROUP BY p1.percentile
    ''',
    con,
)

,percentile,INDUS,NOX
0,10,2.93,0.428
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.07,0.507
4,50,9.69,0.538
5,60,12.83,0.573
6,70,18.10,0.605
7,80,18.10,0.659
8,90,19.58,0.713
9,100,27.74,0.871


In [66]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
